In [ ]:
!pip install timm

A simple and easy to use fastai pipeline

The model definiition below can be modified to handle additional architectures that are available in Timm, but in it's current state will be able to handle efficientnets

In [ ]:
from fastai.vision.all import *
from sklearn.preprocessing import LabelEncoder
import timm
from tqdm import tqdm
from torch.nn import Parameter
from sklearn.model_selection import StratifiedKFold

In [ ]:
data_path = Path('../input/shopee-product-matching')
train_path = data_path/'train_images'
data_path.ls()

In [ ]:
train_df = pd.read_csv(data_path/'train.csv')
train_path = data_path/'train_images'
train_df['image'] = train_df['image'].apply(lambda x: os.path.join(train_path, x))

Since the dataset is imbalanced and we want to ensure that our model is not exposed to a sample in it's valudation that was not in the training set, we'll do the work to get validation indexes for a kfold scheme -- only the first kfold indexes will be used in this example, but the a training loop can be created pretty easily by walking down the list of validation indexes provided from the fxns below

In [ ]:
def kfold_idxs(df, n_splits):
    train_idx, val_idx = [], []
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    for train_index, valid_index in skf.split(df.image, df.label_group):
        train_idx.append(train_index)
        val_idx.append(valid_index)
    return train_idx, val_idx


def get_val_idxs(df, n_splits):
    _, val_idxs = kfold_idxs(df, n_splits)
    return val_idxs

In [ ]:
val_idxs = get_val_idxs(train_df, 2)

In [ ]:
db = DataBlock(blocks = (ImageBlock, CategoryBlock),
              get_x = ColReader('image'),
              get_y = ColReader('label_group'),
              splitter = IndexSplitter(val_idxs[0]),
              item_tfms = Resize(256))

We have now defined our datablock -- this is the template our dataloader will use when creating batches

In [ ]:
dls = db.dataloaders(train_df, bs=64)
dls.show_batch(max_n=3)

Looks like our dataloader is functioning

Lets quickly grab an x and y batch in order to illustrate how our forward pass will function (once we've declared our model)

In [ ]:
xb, yb = next(iter(dls.train))
xb.shape, yb.shape

The model and ArcMarginProduct definition(s) below are borrowed from: https://www.kaggle.com/tanulsingh077/pytorch-metric-learning-pipeline-only-images (Pytorch) which was derived from https://www.kaggle.com/ragnar123/unsupervised-baseline-arcface/notebook (TF)

modified/trimmed down in order to more easily see what's being used in this implementation -- go check those out + head down the rabbit hole to see where they originally derived these from -- they (properly) references the papers/github repos etc

In [ ]:
class ArcFaceNet(nn.Module):
    def __init__(self,
                 n_classes, model_name='efficientnet_b0', s=30.0, 
                 margin=0.50, ls_eps=0.0, theta_zero=0.785, pretrained=True):

        super(ArcFaceNet, self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)
        final_in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity() #could also redefine model without classifier
        self.final = ArcMarginProduct(final_in_features, n_classes,
                                      s=s, m=margin, easy_margin=False, ls_eps=ls_eps)

    def forward(self, x, label):
        feature = self.backbone(x)
        return self.final(feature, label)

In [ ]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features)).to('cuda')
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine,2)).to(cosine.dtype) #needed for to_fp16()
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [ ]:
model = ArcFaceNet(11014).to('cuda');

In [ ]:
#let's make sure our fastai dataloader can do a fwd pass on our x and y batches
model(xb, yb).shape

In [ ]:
#double check that our loss function will work
CrossEntropyLossFlat()(model(xb,yb), yb)

You can uncomment the lines below and see the error that is thrown by just instantiating a learner in the typical fashion

In [ ]:
#learner = Learner(dls, ArcFaceNet(11014, 'efficientnet_b0'),
#                  loss_func=CrossEntropyLossFlat(), metrics=accuracy)#.to_fp16()

Since our learner object normally expects the model to only have one batch in the x forward pass -- we can either manipulate what is defined as x in our datablock/dataloader (ehhhh) -- or change it with a call back during our training loop --> this latter method is what we'll use

Fastai callbacks: https://docs.fast.ai/callback.core.html

In [ ]:
class AmpCallback(Callback):
    def before_batch(self):
        self.learn.xb = (self.x, self.y)

Three lines of code!!! :)

In [ ]:
learner = Learner(dls, ArcFaceNet(11014, 'efficientnet_b0'),
                  loss_func=CrossEntropyLossFlat(), 
                  cbs=[AmpCallback],
                 metrics=accuracy).to_fp16()

In [ ]:
learner.fine_tune(3)

There's a ton of modifications that can be made from data augmentation, to learning rate schduling to mixup etc